In [ ]:
import sys
import numpy
import snappy
from snappy import ProductIO
from snappy import Product
from snappy import Band
from PIL import Image
from jupytep.common.io import *
from jupytep.maps.leaflet.layer import ImageLayer

In [ ]:
jpy = snappy.jpy

# More Java type definitions required for image generation
Color = jpy.get_type('java.awt.Color')
ColorPoint = jpy.get_type('org.esa.snap.core.datamodel.ColorPaletteDef$Point')
ColorPaletteDef = jpy.get_type('org.esa.snap.core.datamodel.ColorPaletteDef')
ImageInfo = jpy.get_type('org.esa.snap.core.datamodel.ImageInfo')
ImageManager = jpy.get_type('org.esa.snap.core.image.ImageManager')
JAI = jpy.get_type('javax.media.jai.JAI')

# Disable JAI native MediaLib extensions 
System = jpy.get_type('java.lang.System')
System.setProperty('com.sun.media.jai.disableMediaLib', 'true')

In [ ]:
file = '/eodata/Sentinel-2/MSI/L1C/2018/05/21/S2B_MSIL1C_20180521T100029_N0206_R122_T33UXT_20180521T120149.SAFE'
product = ProductIO.readProduct(file)
band = product.getBand('B2')
print(band)

In [ ]:
def write_image(band, points, filename, format):
    cpd = ColorPaletteDef(points)
    ii = ImageInfo(cpd)
    band.setImageInfo(ii)
    im = ImageManager.getInstance().createColoredBandImage([band], band.getImageInfo(), 0)
    JAI.create("filestore", im, filename, format)

In [ ]:
points = [ColorPoint(0.05, Color.YELLOW), 
          ColorPoint(0.10, Color.RED), 
          ColorPoint(0.15, Color.BLUE)]
output_file = get_work_dir() + 'snappy_colored_band.png'
write_image(band, points, output_file, 'PNG')

In [ ]:
im = Image.open(output_file)
nx, ny = im.size
im2 = im.resize((int(nx*0.2), int(ny*0.2)), Image.BICUBIC)
output_file_res = get_work_dir() + "snappy_colored_band_res.png"
im2.save(output_file_res)

In [ ]:
layer=ImageLayer()
box=layer.get_bbox(file)
print(box)
png_file_res = "../../../work/snappy_colored_band_res.png"
layer.add_image_layer(png_file_res, box, 'S2_B2_color')
layer.show_layer()